# Linkit Challenge WS 2022/23 Hand Gesture Detection - Team X

## 1. Setup
`pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies`

## 2. Annotation

## 3. Training

#### 3.1. Import Packages

In [1]:
import pytorch_lightning as pl # PyTorch Lightning for easier training and evaluation of models
import torch # PyTorch
import cv2 as cv2 # OpenCV for image processing
import matplotlib.pyplot as plt # for plotting
import matplotlib.patches as patches  # for plotting bounding boxes
%matplotlib inline
import uuid   # Unique identifier
import os # File system operations
import time # Time operations


/Users/philippkiesling/.conda/envs/linkit_hand_gesture_recognition/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 3.2. Load and Test Pretrained Model
In Computer Vision, we usually use pretrained Models, to reduce the number of samples required for training. In this case, we use a pretrained YOLOv5 model, which was trained on the COCO dataset. The COCO dataset contains 80 different classes, which are not relevant for our task. For now, let's just test the model on a random image.

#### Tasks:
_**Task 3.1:**_ Can you find better models for our task?
_**Task 3.2:**_ Can you find better pretrained weights for our task?
_Note:_ You can find possible models and weights on:
1. huggingface [Model Hub](https://huggingface.co/models?pipeline_tag=object-detection&sort=downloads).
2. pytorch [Model Zoo](https://pytorch.org/docs/stable/torchvision/models.html).
3. pytorch [Model Hub](https://pytorch.org/hub/).


In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /Users/philippkiesling/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-12-5 Python-3.10.8 torch-1.13.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


We Load  an Image and run the model on it. The model returns a list of bounding boxes, with their corresponding class and confidence score.

In [4]:
# Load image
img = cv2.imread('datasets/example/zidane.png')

In [5]:
results = model(img)

In [6]:
# Display image and Convert to RGB, display labels and bounding boxes from the results with cv2
fig, ax = plt.subplots()
ax.imshow(img[:,:,::-1])

# Draw bounding boxes and labels of detections
for *rect,  conf, name, cls   in results.pandas().xyxy[0].values:
    x1, y1, x2, y2 = rect
    rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.text(x1, y1, f'{cls} {conf:.2f}', fontsize=12, c='white')
plt.show()

#### 3.3 Adapt the Model for our Task
We need to adapt the model for our task. Therefore, we need to remove the last layer of the model, and replace it with a new layer, which only contains 3 classes (one for each hand gesture).
Since we use the [yolov5 implementation of ultralytics](https://github.com/ultralytics/yolov5), we can use their provided training script to train our model.


First we define our 3 label names, and the path to the images and labels.

This is done in the dataset.yaml file.
Here we have to define the 3 classes, and the path to the images and labels.
Make sure to use the correct mapping between classID and label
1. 0 -> "rock"
2. 1 -> "paper"
3. 2 -> "scissors"

Now we train our model.
we use the dataset.yml file to define the path to the images and labels, and the number of classes.

For different Parameter Configurations refer to:
https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data

In [8]:
# Train YOLOv5s on COCO128 for 3 epochs
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 3 --data ../dataset.yaml --weights yolov5s.pt --cache --workers 0 --save_period 1

wandb: Currently logged in as: philippkiesling. Use `wandb login --relogin` to force relogin
usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--seed SEED]
                [--local_rank LOCAL_RANK] [--entity ENTITY]
                [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval 